In [1]:
import pystac
import matplotlib.pyplot as plt
from mccn.extent import GeoBoxBuilder
from mccn._types import FilterConfig
from mccn.loader.raster import RasterLoader
from mccn.parser import parse_item, band_filter

In [4]:
rea_collection = pystac.Collection.from_file(
    "tests/files/unit_tests/raster/stac/std/collection.json"
)
items = [parse_item(item) for item in rea_collection.get_items(recursive=True)]
pitems = [] 
for item in items: 
    pitem = band_filter(item, ["price"])
    if pitem: 
        pitems.append(pitem)

rea_loader = RasterLoader(
    pitems,
    filter_config=FilterConfig(
        GeoBoxBuilder.from_collection(rea_collection, 1000)
    ),
)
rea_ds = rea_loader.load()
rea_ds

<xarray.Dataset> Size: 4MB
Dimensions:      (lat: 1000, lon: 1000, time: 1)
Coordinates:
  * lat          (lat) float64 8kB -37.54 -37.54 -37.54 ... -38.0 -38.0 -38.0
  * lon          (lon) float64 8kB 144.4 144.4 144.4 144.4 ... 145.0 145.0 145.0
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 8B 2024-12-31T14:20:00
Data variables:
    price        (time, lat, lon) float32 4MB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0